# Time consistency in ERA5

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
start = "1940-01"
stop = None  # None: present

## Define request

In [ ]:
collection_id = "reanalysis-era5-pressure-levels-monthly-means"
variable = "temperature"
request = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": [
        "temperature",
        "u_component_of_wind",
        "vertical_velocity",
        "geopotential",
        "ozone_mass_mixing_ratio",
        "relative_humidity",
        "fraction_of_cloud_cover",
    ],
    "pressure_level": [
        "1",
        "2",
        "3",
        "5",
        "7",
        "10",
        "20",
        "30",
        "50",
        "70",
        "100",
        "125",
        "150",
        "175",
        "200",
        "225",
        "250",
        "300",
        "350",
        "400",
        "450",
        "500",
        "550",
        "600",
        "650",
        "700",
        "750",
        "775",
        "800",
        "825",
        "850",
        "875",
        "900",
        "925",
        "950",
        "975",
        "1000",
    ],
    "time": "00:00",
}
requests = download.update_request_date(request, start=start, stop=stop)

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    transform_func=diagnostics.spatial_weighted_mean,
    chunks={"year": 1, "variable": 1},
)
# Convert plev to hPa
with xr.set_options(keep_attrs=True):
    ds["plev"] = ds["plev"] / 100
    ds["z"] /= 9.8
ds["plev"].attrs["units"] = "hPa"
ds["z"].attrs["units"] = "m"

## Compute anomaly

In [ ]:
group = "forecast_reference_time.month"
with xr.set_options(keep_attrs=True):
    ds_anoma = ds.groupby(group) - ds.groupby(group).mean()
for varname, da in ds_anoma.data_vars.items():
    da.attrs["long_name"] += " anomaly"

## Show min and max values

In [ ]:
datasets = []
for reduction in ("min", "max"):
    datasets.append(
        getattr(ds_anoma.sel(plev=slice(1000, 10)), reduction)().expand_dims(
            reduction=[reduction]
        )
    )
df = xr.concat(datasets, "reduction").to_pandas()
df.T

## Plot Hovmöller diagrams

In [ ]:
plot_dict = {
    "t": {
        "levels": np.arange(-5, 5.5, 0.5),
        "cmap": "RdBu_r",
        "yscale": "log",
    },
    "u": {
        "levels": np.arange(-20, 20 + 2, 2),
        "cmap": "PuOr",
        "yscale": "log",
    },
    "w": {
        "levels": np.arange(-0.8e-3, 0.8e-3 + 0.8e-4, 0.8e-4),
        "cmap": "PuOr",
        "yscale": "linear",
    },
    "z": {
        "levels": np.arange(-300, 300 + 30, 30),
        "cmap": "seismic",
        "yscale": "log",
    },
    "o3": {
        "levels": np.arange(-1.0e-6, 1.0e-6 + 1.0e-7, 1.0e-7),
        "cmap": "RdGy_r",
        "yscale": "log",
    },
    "r": {
        "levels": np.arange(-5, 5 + 0.5, 0.5),
        "cmap": "BrBG",
        "yscale": "linear",
    },
    "cc": {
        "levels": np.arange(-0.01, 0.01 + 0.001, 0.001),
        "cmap": "PRGn",
        "yscale": "linear",
    },
}

eruptions = (
    {
        "volcano": "Pinatubo",
        "date": "1991-06-15",
    },
    {
        "volcano": "El Chichón, Mount St. Helens",
        "date": "1981-01-01",
        "linestyle": "--",
    },
    {
        "volcano": "Agung",
        "date": "1963-02-24",
    },
    {
        "volcano": "Bezymianny",
        "date": "1956-01-01",
    },
)

zooms = {
    "Pinatubo": slice("1988-01-01", "1995-12-31"),
    "Agung": slice("1962-01-01", "1967-12-31"),
    "El Chicon": slice("1980-01-01", "1985-12-31"),
    "Entire period": slice("1940-01-01", "2022-12-31"),
}

for title, zoom in zooms.items():
    ds_to_plot = ds_anoma.sel(forecast_reference_time=zoom)
    fig, axs = plt.subplots(len(ds_anoma.data_vars), figsize=(15, 24))
    for (varname, kwargs), ax in zip(plot_dict.items(), axs):
        ds_to_plot[varname].plot.contourf(
            y="plev", yincrease=False, extend="both", ax=ax, **kwargs
        )
        for eruption in eruptions:
            date = np.datetime64(eruption["date"])
            if (
                not ds_to_plot["forecast_reference_time"].min()
                <= date
                <= ds_to_plot["forecast_reference_time"].max()
            ):
                continue
            ax.axvline(
                date,
                color="k",
                lw=0.5,
                **{k: v for k, v in eruption.items() if k not in ["volcano", "date"]}
            )
            ax.text(
                date,
                ax.get_ylim()[0],
                " " + eruption["volcano"],
                rotation="vertical",
                ha="left",
                va="bottom",
            )
    fig.suptitle(title, fontsize=16, x=0.45, y=0.92)